In [7]:
import pandas as pd
import numpy as np

In [10]:
select * from public.venue

,venueid,venuename,venuecity,venuestate,venueseats
0,1,Toyota Park,Bridgeview,IL,0.0
1,3,RFK Stadium,Washington,DC,0.0
2,6,New York Giants Stadium,East Rutherford,NJ,80242.0
3,8,The Home Depot Center,Carson,CA,0.0
4,11,Robertson Stadium,Houston,TX,0.0
...,...,...,...,...,...
197,255,Venetian Hotel,Las Vegas,NV,NaN
198,258,Tropicana Hotel,Las Vegas,NV,NaN
199,300,Kennedy Center Opera House,Washington,DC,0.0
200,306,Lyric Opera House,Baltimore,MD,0.0


In [17]:
select * from public.sales

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,97197,110942,43282,515,7003,1827,2,708.0,106.20,2008-01-01 02:30:52+00:00
1,110328,126347,6955,394,6213,1827,1,347.0,52.05,2008-01-01 01:00:19+00:00
2,165890,222879,2186,664,6870,1827,2,4192.0,628.80,2008-01-01 12:59:34+00:00
3,65082,73790,20429,451,1150,1827,4,472.0,70.80,2008-01-01 06:06:57+00:00
4,7011,7613,5933,1503,4515,1828,1,177.0,26.55,2008-01-02 01:52:35+00:00
...,...,...,...,...,...,...,...,...,...,...
172451,157586,206655,13213,47394,3665,2191,2,4018.0,602.70,2008-12-31 10:01:25+00:00
172452,154069,178359,3897,5815,8168,2191,1,290.0,43.50,2008-12-31 04:28:40+00:00
172453,3737,4035,11666,23692,6034,2191,2,302.0,45.30,2008-12-31 04:53:26+00:00
172454,37817,42124,24592,26543,6783,2191,4,1300.0,195.00,2008-12-31 04:13:19+00:00


In [7]:
select * from public.event

,eventid,venueid,catid,dateid,eventname,starttime
0,423,303,8,1827,La Damnation de Faust,2008-01-01 19:30:00+00:00
1,2099,211,7,1827,The Bacchae,2008-01-01 19:30:00+00:00
2,6640,38,9,1827,Herbie Hancock,2008-01-01 19:30:00+00:00
3,8621,9,9,1827,Hot Chip,2008-01-01 19:00:00+00:00
4,1376,235,6,1827,The King and I,2008-01-01 14:30:00+00:00
...,...,...,...,...,...,...
8793,6034,45,9,2191,War,2008-12-31 14:00:00+00:00
8794,6783,60,9,2191,The Police,2008-12-31 15:00:00+00:00
8795,6857,18,9,2191,Judas Priest,2008-12-31 14:00:00+00:00
8796,7192,54,9,2191,Lindsey Buckingham,2008-12-31 19:30:00+00:00


## How do venues effect ticket sales? 

First, let's look at the top 10 and bottom 10 venues by total revenue (calculated here by summing the price paid column by venue), limiting the query to 10, and ordering by revenue. 

In [4]:
select 
v.venueid, 
coalesce(sum(s.qtysold),0) as num_tickets_sold, 
coalesce(sum(s.pricepaid),0) as venue_revenue, 
count(distinct s.salesid) as num_sales 
from public.venue v
left join public.event e 
on v.venueid = e.venueid
left join public.sales s
on e.eventid = s.eventid
group by v.venueid
order by venue_revenue desc

limit 10

,venueid,num_tickets_sold,venue_revenue,num_sales
0,220,3326,1115182.0,1672
1,203,3187,1032156.0,1583
2,216,2948,978765.0,1456
3,238,2838,939257.0,1451
4,226,2934,936312.0,1465
5,222,2549,894275.0,1265
6,205,2828,891172.0,1413
7,217,2999,885686.0,1488
8,218,2702,877993.0,1357
9,248,2502,857031.0,1244


In [3]:
select 
v.venueid, 
sum(s.qtysold) as num_tickets_sold, 
sum(s.pricepaid) as venue_revenue, 
count(distinct s.salesid) as num_sales 
from public.venue v
left join public.event e 
on v.venueid = e.venueid
left join public.sales s
on e.eventid = s.eventid
group by v.venueid
order by venue_revenue asc

limit 10

,venueid,num_tickets_sold,venue_revenue,num_sales
0,251,634,192899.0,309
1,262,658,196406.0,332
2,256,758,233042.0,378
3,261,660,234628.0,327
4,250,751,242200.0,378
5,254,862,253371.0,427
6,260,787,260292.0,398
7,59,728,262091.0,372
8,252,862,274994.0,418
9,258,905,281683.0,443


In [5]:
top_10_venues_list = top10_venues['venueid']
bottom_10_venues_list = bottom_10_venues['venueid']

In [8]:
# Create a DataFrame for top 10 venues with a 'class' column set to 'T'
top_venues_df = pd.DataFrame(top_10_venues_list, columns=['venueid'])
top_venues_df['class'] = 'T'

# Create a DataFrame for bottom 10 venues with a 'class' column set to 'B'
bottom_venues_df = pd.DataFrame(bottom_10_venues_list, columns=['venueid'])
bottom_venues_df['class'] = 'B'

# Concatenate the two DataFrames
venues_df = pd.concat([top_venues_df, bottom_venues_df], ignore_index=True)
venues_df

,venueid,class
0,220,T
1,203,T
2,216,T
3,238,T
4,226,T
5,222,T
6,205,T
7,217,T
8,218,T
9,248,T


In [11]:
# Assuming there's a DataFrame named `venues` with venue details that you want to join with `venues_df`
merged_df = pd.merge(venues_df, venues, on='venueid', how='left')
merged_df

,venueid,class,venuename,venuecity,venuestate,venueseats
0,220,T,Lunt-Fontanne Theatre,New York City,NY,0.0
1,203,T,August Wilson Theatre,New York City,NY,0.0
2,216,T,Helen Hayes Theatre,New York City,NY,0.0
3,238,T,Winter Garden Theatre,New York City,NY,0.0
4,226,T,Nederlander Theatre,New York City,NY,0.0
5,222,T,Majestic Theatre,New York City,NY,0.0
6,205,T,Ethel Barrymore Theatre,New York City,NY,0.0
7,217,T,Hilton Theatre,New York City,NY,0.0
8,218,T,Imperial Theatre,New York City,NY,0.0
9,248,T,Charles Playhouse,Boston,MA,0.0


From here, we can see that most of the top 10 venues by revenue are in New York City and the bottom 10 venues are mostly in Las Vegas. 

## How do user interests effect sales?

In [42]:
select * from public.users

,userid,username,firstname,lastname,city,state,email,phone,likesports,liketheatre,likeconcerts,likejazz,likeclassical,likeopera,likerock,likevegas,likebroadway,likemusicals
0,2,PGL08LJI,Vladimir,Humphrey,Murfreesboro,SK,Suspendisse.tristique@nonnisiAenean.edu,(783) 492-1886,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,0.0,1.0
1,4,XDZ38RDD,Barry,Roy,Omaha,AB,sed@lacusUtnec.ca,(355) 452-8168,0.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
2,5,AEB55QTM,Reagan,Hodge,Forest Lake,NS,Cum@accumsan.com,(476) 519-9131,NaN,NaN,1.0,0.0,NaN,NaN,1.0,1.0,0.0,1.0
3,7,OWY35QYB,Tamekah,Juarez,Moultrie,WV,elementum@semperpretiumneque.ca,(297) 875-7247,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,0.0,0.0
4,9,MSD36KVR,Mufutau,Watkins,Port Orford,MD,Integer.mollis.Integer@tristiquealiquet.org,(725) 719-7670,1.0,0.0,NaN,0.0,1.0,NaN,NaN,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49985,49983,RCS76EGO,Colorado,Fuller,Blacksburg,SK,dolor.sit@acmattisvelit.com,(894) 748-8662,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,0.0,NaN
49986,49984,FFP96MVH,Caryn,Cabrera,Beckley,CO,ac.turpis.egestas@scelerisquelorem.ca,(641) 730-2786,1.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN
49987,49986,WSZ25RSI,Basil,Sexton,Milwaukee,CO,sociis.natoque.penatibus@vitae.org,(858) 335-0367,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0
49988,49989,RRS41ZTJ,Evangeline,Franco,Bakersfield,NT,nibh.enim@egestas.ca,(976) 963-7780,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,1.0


Earlier, we created subsets that look at users who answered yes to the interests.

In [43]:
classical_fans = users[users['likeclassical'] == 1]
opera_fans = users[users['likeopera'] == 1]
rock_fans = users[users['likerock'] == 1]
vegas_fans = users[users['likevegas'] == 1]
broadway_fans = users[users['likebroadway'] == 1]
musicals_fans = users[users['likemusicals'] == 1]

In [44]:
classical_fans

,userid,username,firstname,lastname,city,state,email,phone,likesports,liketheatre,likeconcerts,likejazz,likeclassical,likeopera,likerock,likevegas,likebroadway,likemusicals
0,2,PGL08LJI,Vladimir,Humphrey,Murfreesboro,SK,Suspendisse.tristique@nonnisiAenean.edu,(783) 492-1886,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,0.0,1.0
3,7,OWY35QYB,Tamekah,Juarez,Moultrie,WV,elementum@semperpretiumneque.ca,(297) 875-7247,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,0.0,0.0
4,9,MSD36KVR,Mufutau,Watkins,Port Orford,MD,Integer.mollis.Integer@tristiquealiquet.org,(725) 719-7670,1.0,0.0,NaN,0.0,1.0,NaN,NaN,NaN,0.0,1.0
8,18,VDP05MXU,Germaine,Valdez,Kokomo,WY,cursus.Integer@arcuVestibulumante.com,(998) 879-8668,NaN,1.0,1.0,NaN,1.0,1.0,NaN,0.0,1.0,1.0
9,19,CXQ97IWP,Amal,Landry,Lomita,NT,euismod@turpis.org,(891) 526-1468,NaN,0.0,1.0,NaN,1.0,NaN,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49960,49935,AMY61JPQ,Risa,Villarreal,Kingston,MI,at@nostraper.edu,(736) 489-8963,NaN,NaN,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN
49963,49939,JQI41JZD,Cathleen,Frazier,Lawrence,NT,vestibulum@eget.ca,(259) 483-8068,0.0,NaN,0.0,1.0,1.0,0.0,NaN,NaN,0.0,1.0
49972,49957,HJR72LGP,Nita,Simpson,Fajardo,QC,vitae.purus@necimperdietnec.com,(114) 818-7413,1.0,0.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0
49978,49972,WXR91PLA,Wesley,Delaney,Gainesville,NL,ligula.eu.enim@metus.edu,(355) 243-0690,NaN,NaN,1.0,0.0,1.0,1.0,NaN,0.0,NaN,NaN


In [45]:
select * from public.sales where sellerid = 82

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,5318,5719,82,809,4659,1946,1,54.0,8.1,2008-04-30 03:51:09+00:00
1,5317,5719,82,6300,4659,1953,2,108.0,16.2,2008-05-07 03:48:49+00:00
2,154438,178821,82,3341,4158,2090,1,334.0,50.1,2008-09-21 10:45:12+00:00


Let's make some variables to hold our key stats and see how the classical group measures up. We are going to speed this up by creating a function to use on all the dataframes. 

In [46]:
def aggregate_fan_sales(fan_df, sales_df):
    """
    Merges a fan dataframe with the sales dataframe on userid matching either buyerid or sellerid
    and performs the required aggregations.
    
    Parameters:
    - fan_df: DataFrame of fans with at least 'userid' column
    - sales_df: DataFrame of sales
    
    Returns:
    - DataFrame of aggregated sales data for each user in fan_df
    """
    merged_df = fan_df.merge(sales_df, left_on='userid', right_on='buyerid', how='left') \
                      .append(fan_df.merge(sales_df, left_on='userid', right_on='sellerid', how='left')) \
                      .drop_duplicates()

    aggregated_data = merged_df.groupby('userid').agg(
        total_qty_sold=('qtysold', 'sum'),
        total_price_paid=('pricepaid', 'sum'),
        unique_sales_count=('salesid', pd.Series.nunique)
    ).reset_index()

    return aggregated_data

# Example usage for different fan groups
classical_fans_aggregated = aggregate_fan_sales(classical_fans, sales)
opera_fans_aggregated = aggregate_fan_sales(opera_fans, sales)
rock_fans_aggregated = aggregate_fan_sales(rock_fans, sales)
vegas_fans_aggregated = aggregate_fan_sales(vegas_fans, sales)
broadway_fans_aggregated = aggregate_fan_sales(broadway_fans, sales)
musicals_fans_aggregated = aggregate_fan_sales(musicals_fans, sales)

In [52]:
classical_qty = classical_fans_aggregated['total_qty_sold'].sum()
classical_fans = classical_fans_aggregated['userid'].nunique()
classical_total_paid = classical_fans_aggregated['total_price_paid'].sum()

opera_qty = opera_fans_aggregated['total_qty_sold'].sum()
opera_fans = opera_fans_aggregated['userid'].nunique()
opera_total_paid = opera_fans_aggregated['total_price_paid'].sum()

rock_qty = rock_fans_aggregated['total_qty_sold'].sum()
rock_fans = rock_fans_aggregated['userid'].nunique()
rock_total_paid = rock_fans_aggregated['total_price_paid'].sum()

vegas_qty = vegas_fans_aggregated['total_qty_sold'].sum()
vegas_fans = vegas_fans_aggregated['userid'].nunique()
vegas_total_paid = vegas_fans_aggregated['total_price_paid'].sum()

broadway_qty = broadway_fans_aggregated['total_qty_sold'].sum()
broadway_fans = broadway_fans_aggregated['userid'].nunique()
broadway_total_paid = broadway_fans_aggregated['total_price_paid'].sum()

musicals_qty = musicals_fans_aggregated['total_qty_sold'].sum()
musicals_fans = musicals_fans_aggregated['userid'].nunique()
musicals_total_paid = musicals_fans_aggregated['total_price_paid'].sum()

In [53]:
total_users = 49990 
total_paid = sales['pricepaid'].sum()
total_tickets = sales['qtysold'].sum()
# Calculating percentages for each group

# Classical
classical_fans_perc = classical_fans / total_users * 100
classical_total_paid_perc = classical_total_paid / total_paid * 100
classical_total_tix_perc = classical_qty / total_tickets * 100

# Opera
opera_fans_perc = opera_fans / total_users * 100
opera_total_paid_perc = opera_total_paid / total_paid * 100
opera_total_tix_perc = opera_qty / total_tickets * 100

# Rock
rock_fans_perc = rock_fans / total_users * 100
rock_total_paid_perc = rock_total_paid / total_paid * 100
rock_total_tix_perc = rock_qty / total_tickets * 100

# Vegas
vegas_fans_perc = vegas_fans / total_users * 100
vegas_total_paid_perc = vegas_total_paid / total_paid * 100
vegas_total_tix_perc = vegas_qty / total_tickets * 100

# Broadway
broadway_fans_perc = broadway_fans / total_users * 100
broadway_total_paid_perc = broadway_total_paid / total_paid * 100
broadway_total_tix_perc = broadway_qty / total_tickets * 100

# Musicals
musicals_fans_perc = musicals_fans / total_users * 100
musicals_total_paid_perc = musicals_total_paid / total_paid * 100
musicals_total_tix_perc = musicals_qty / total_tickets * 100

In [38]:
print(f'Fans Percentage from Total: ')
print(f'Classical: {classical_fans_perc} ')
print(f'Rock: {rock_fans_perc} ')
print(f'Vegas: {vegas_fans_perc} ')
print(f'Broadway: {broadway_fans_perc} ')
print(f'Musicals: {musicals_fans_perc} ')

Fans Percentage from Total: 
Classical: 25.05101020204041 
Rock: 24.76495299059812 
Vegas: 24.832966593318666 
Broadway: 24.782956591318264 
Musicals: 25.219043808761754 


In [58]:
import pandas as pd

# Assuming classical_fans_aggregated is missing and needs to be created for consistency
# This is a placeholder for how you might calculate it, similar to the other categories
# classical_fans_aggregated = ...

# Calculate the total for each category
total_fans = 49990 
total_paid = sales['pricepaid'].sum()
total_tix = sales['qtysold'].sum()

# Calculate percentages for each category
categories = ['classical', 'rock', 'vegas', 'broadway', 'musicals']
fans_perc = []
total_paid_perc = []
total_tix_perc = []

for category in categories:
    df = eval(f"{category}_fans_aggregated")  # Dynamically use the dataframe
    fans_perc.append(df['userid'].count() / total_fans * 100)
    total_paid_perc.append(df['total_price_paid'].sum() / total_paid * 100)
    total_tix_perc.append(df['total_qty_sold'].sum() / total_tix * 100)

# Adding totals/sum all as a category
fans_perc.append(100)  # Since it's the total, it's 100%
total_paid_perc.append(100)
total_tix_perc.append(100)
categories.append('total/sum all')

# Create the dataframe
stats_df = pd.DataFrame({
    'category': categories,
    'fans_perc': fans_perc,
    'total_paid_perc': total_paid_perc,
    'total_tix_perc': total_tix_perc
})

stats_df

,category,fans_perc,total_paid_perc,total_tix_perc
0,classical,25.051010,49.494483,49.873896
1,rock,24.764953,50.508173,50.259882
2,vegas,24.832967,49.403270,49.529896
3,broadway,24.782957,49.942812,49.619660
4,musicals,25.219044,50.526258,50.441727
5,total/sum all,100.000000,100.000000,100.000000


Among the different music interests of users, the number of fans, tickets sold, and total amount paid are roughly similar, so we cannot conclude that there is a difference in sales between interests. 